#Exercise Session 1: Overview of the Xpress python Interface

###Creating a simple LP

>$min\ x + y$ \\
>$ s.t.\ 2x + 3y \geq 6$ \\
>$ 4x + 2y \geq 7$ \\
>$ x,y \geq 0$ \\
###Also:
***Knapsack Problem***

***NumPy operators and arrays of variables constraints*** \\

***Production planning with quadratic cost***



Exercise is a part of CO@Work Summer School 2020 by ZIB.

In [1]:
pip install xpress

     |████████████████████████████████| 43.8MB 95kB/s 


In [2]:
import xpress as xp
print(xp.manual())
print(xp.examples())

Using the Community license in this session. If you have a full Xpress license, first set the XPAUTH_PATH environment variable to the full path to your license file, xpauth.xpr, and then restart Python. If you want to use the FICO Community license and no longer want to see this message, set the XPAUTH_PATH environment variable to: /usr/local/lib/python3.6/dist-packages/xpress/license/community-xpauth.xpr
NB: setting XPAUTH_PATH will also affect any other Xpress products installed on your system.
/usr/local/lib/python3.6/dist-packages/xpress/doc/python-interface.pdf
/usr/local/lib/python3.6/dist-packages/xpress/examples/


In [3]:
x = xp.var(name='myvar_1')
y = xp.var(name='myvar_2') #variables by defualt are non-negative

In [4]:
constr1 = 2*x + 3*y >= 6
constr2 = 4*x + 2*y >= 7
obj = x + y

In [5]:
p = xp.problem(x,y, #<-- variabes(always first to be added)
               constr1,constr2,   #<-- constraints and objectives (in no order)
               obj)

In [6]:
p.solve()
print(f'solution: x={p.getSolution(x)}, y ={p.getSolution(y)}')

FICO Xpress v8.11.0, Community, solve started 8:39:37, Nov 24, 2020
Heap usage: 338KB (peak 338KB, 593KB system)
Minimizing LP noname with these control settings:
OUTPUTLOG = 1

Original problem has:
         2 rows            2 cols            4 elements
Presolved problem has:
         2 rows            2 cols            4 elements
Presolve finished in 0 seconds
Heap usage: 339KB (peak 351KB, 595KB system)

Coefficient range                    original                 solved        
  Coefficients   [min,max] : [ 2.00e+00,  4.00e+00] / [ 5.00e-01,  1.50e+00]
  RHS and bounds [min,max] : [ 6.00e+00,  7.00e+00] / [ 1.75e+00,  3.00e+00]
  Objective      [min,max] : [ 1.00e+00,  1.00e+00] / [ 1.00e+00,  1.00e+00]
Autoscaling applied standard scaling

 
   Its         Obj Value      S   Ninf  Nneg   Sum Dual Inf  Time
     0           .000000      D      2     0        .000000     0
     2          2.375000      D      0     0        .000000     0
Uncrunching matrix
Optimal solution found


In [7]:
#add constraint x + 2y <= 2 to the problem and then resolve. The problem will now be infeasible

In [8]:
constr3 = x + 2*y <= 2

p.addConstraint(constr3)

p.solve()

FICO Xpress v8.11.0, Community, solve started 8:39:37, Nov 24, 2020
Heap usage: 2605KB (peak 2607KB, 595KB system)
Minimizing LP noname with these control settings:
OUTPUTLOG = 1

Original problem has:
         3 rows            2 cols            6 elements
 
 
The problem is infeasible due to row R3
Presolve finished in 0 seconds
Heap usage: 2605KB (peak 2607KB, 595KB system)


In [9]:
#use the iisall function to understand what made the problem infeasible. If needed, type help(p.iisall) for its docstring.

In [10]:
p.iisall()

   Name      Type    Sense    Bound 
 R1          row     GE      6.000000
 R3          row     LE      2.000000
 myvar_2     column  LO       .000000
Found IIS 1 (2 rows and 1 bounds)
 
No more IIS can be found


Now delete the constraint (using the `problem.delConstraint` method) and, just to avoid the same solve as before, change the objective function coefficient of $x$ to 4. Then re-solve and print primal and dual solution using `problem.getSolution` for the primal solution and `problem.getDual` for the dual solution. Recall that these functions can be called without arguments to obtain the whole vector, or with a variable/constraint object to get one or more values.

In [11]:
p.delConstraint(constr3)
p.chgobj([x],[4])
p.solve()

print(f'Solution: x={p.getSolution(x)}, y={p.getSolution(y)}')
print(f'Dual: constr1->{p.getDual(constr1)}, constr2->{p.getDual(constr2)}')

FICO Xpress v8.11.0, Community, solve started 8:39:37, Nov 24, 2020
Heap usage: 4956KB (peak 4956KB, 832KB system)
Minimizing LP noname with these control settings:
OUTPUTLOG = 1

Original problem has:
         2 rows            2 cols            4 elements
Presolved problem has:
         2 rows            2 cols            4 elements
Presolve finished in 0 seconds
Heap usage: 4956KB (peak 4956KB, 832KB system)

Coefficient range                    original                 solved        
  Coefficients   [min,max] : [ 2.00e+00,  4.00e+00] / [ 5.00e-01,  1.50e+00]
  RHS and bounds [min,max] : [ 6.00e+00,  7.00e+00] / [ 1.75e+00,  3.00e+00]
  Objective      [min,max] : [ 1.00e+00,  4.00e+00] / [ 1.00e+00,  4.00e+00]
Autoscaling applied standard scaling

 
   Its         Obj Value      S   Ninf  Nneg   Sum Dual Inf  Time
     0          5.750000      D      0     1       1.000000     0
     1          3.500000      D      0     0        .000000     0
Uncrunching matrix
Optimal solution fo

#Knapsack problem:

Formulate and solve a knapsack problem with the following vaue/weight vectors:

>$ v = (12, 15, 9, 11, 8, 7, 5)$ \\
>$ w = (13, 18, 9, 12, 8, 10, 4)$ \\

with knapsack capacity $ C = 40$ \\
You will need `xpress.Sum` operator. The formuation of a knapsack problem is as follows:

$$
\begin{array}{lllrrrrrrr}
\max & \sum_{i=1}^n v_i x_i\\
\textrm{s.t.} & \sum_{i=1}^n w_i x_i \le C\\
& x_i \in \{0,1\} \forall i=1,\ldots{},n
\end{array}
$$




In [12]:
v = [12,15,9,11,8,7,5]
w = [13,18,9,12,8,10,4]
C = 40

n = len(v)
x = [xp.var(vartype = xp.binary) for _ in range (n)]

k_con = xp.Sum(w[i]*x[i] for i in range (n)) <= C
k_obj = xp.Sum(v[i]*x[i] for i in range (n))

p = xp.problem(x,k_con,k_obj, sense = xp.maximize) # optimization sense is min by default

p.solve()

print(p.getSolution())


FICO Xpress v8.11.0, Community, solve started 8:39:37, Nov 24, 2020
Heap usage: 338KB (peak 338KB, 562KB system)
Maximizing MILP noname with these control settings:
OUTPUTLOG = 1

Original problem has:
         1 rows            7 cols            7 elements         7 globals
Presolved problem has:
         1 rows            7 cols            7 elements         7 globals
Presolve finished in 0 seconds
Heap usage: 364KB (peak 369KB, 564KB system)

Coefficient range                    original                 solved        
  Coefficients   [min,max] : [ 4.00e+00,  1.80e+01] / [ 2.50e-01,  1.12e+00]
  RHS and bounds [min,max] : [ 1.00e+00,  4.00e+01] / [ 1.00e+00,  2.50e+00]
  Objective      [min,max] : [ 5.00e+00,  1.50e+01] / [ 5.00e+00,  1.50e+01]
Autoscaling applied standard scaling

Will try to keep branch and bound tree memory usage below 11.9GB
 *** Heuristic solution found:      .000000      Time: 0 ***
 *** Heuristic solution found:    31.000000      Time: 0 ***
Starting concurr

### NumPy operators and arrays of variables/constraints

NumPy is an essential toolbox for Python users. The Xpress Python interface can handle (multi-)arrays of floats, variables, expressions, constraints as naturally as with lists thereof.

In [13]:
import numpy as np

#The folowing two statements create two equivalent NumPy arrays of variables

x = np.array([xp.var(vartype = xp.binary, name = 'x({})'.format(i)) for i in range(20)], dtype = xp.npvar)
y = xp.vars(20,     vartype = xp.binary, name = 'x')

#Exercise:
We can rewrite the knapsack problem using NumPy constructs

In [14]:
import xpress as xp
import numpy as np

v = np.array([12,15,9,11,8, 7,5])
w = np.array([13,18,9,12,8,10,4])
C = 40

n = len(v)
x = xp.vars(n, vartype=xp.binary)

In [15]:
# xp.Dot only works with numpy arrays
k_con = xp.Dot(w,x) <= C
k_obj = xp.Dot(v,x)

p = xp.problem(x, k_con, k_obj, sense=xp.maximize)

p.solve()

print(p.getSolution())


FICO Xpress v8.11.0, Community, solve started 8:39:38, Nov 24, 2020
Heap usage: 338KB (peak 338KB, 623KB system)
Maximizing MILP noname with these control settings:
OUTPUTLOG = 1

Original problem has:
         1 rows            7 cols            7 elements         7 globals
Presolved problem has:
         1 rows            7 cols            7 elements         7 globals
Presolve finished in 0 seconds
Heap usage: 364KB (peak 369KB, 625KB system)

Coefficient range                    original                 solved        
  Coefficients   [min,max] : [ 4.00e+00,  1.80e+01] / [ 2.50e-01,  1.12e+00]
  RHS and bounds [min,max] : [ 1.00e+00,  4.00e+01] / [ 1.00e+00,  2.50e+00]
  Objective      [min,max] : [ 5.00e+00,  1.50e+01] / [ 5.00e+00,  1.50e+01]
Autoscaling applied standard scaling

Will try to keep branch and bound tree memory usage below 11.9GB
 *** Heuristic solution found:      .000000      Time: 0 ***
 *** Heuristic solution found:    31.000000      Time: 0 ***
Starting concurr

#Exercise:

Create a random 20x30 numpy matrix $A$ and vectors $b$ and $c$, then create a problem with $c$ as objective function coefficient vector and constraints $Ax \leq b.$

In [16]:
n = 30 #number of variables
m = 20 #number of constraints

A = np.random.random((m,n))
b = np.random.random(m)
c = np.random.random(n)

x = xp.vars(n)

objective = xp.Dot(c, x)
consys = xp.Dot(A, x) <= b

p = xp.problem(x,objective, consys, sense = xp.maximize)

p.solve()


FICO Xpress v8.11.0, Community, solve started 8:39:38, Nov 24, 2020
Heap usage: 356KB (peak 356KB, 625KB system)
Maximizing LP noname with these control settings:
OUTPUTLOG = 1

Original problem has:
        20 rows           30 cols          600 elements
Presolved problem has:
        20 rows           30 cols          600 elements
Presolve finished in 0 seconds
Heap usage: 367KB (peak 389KB, 627KB system)

Coefficient range                    original                 solved        
  Coefficients   [min,max] : [ 5.58e-04, 10.00e-01] / [ 5.58e-04, 10.00e-01]
  RHS and bounds [min,max] : [ 6.08e-03,  9.81e-01] / [ 6.08e-03,  9.81e-01]
  Objective      [min,max] : [ 6.73e-02,  9.61e-01] / [ 6.73e-02,  9.61e-01]
Autoscaling applied standard scaling

 
   Its         Obj Value      S   Ninf  Nneg   Sum Dual Inf  Time
     0           .387407      D     12     0        .000000     0
     5           .069425      D      0     0        .000000     0
Uncrunching matrix
Optimal solution found

#Exercise:

Make the objective function quadratic, with a matrix $Q$ that is PSD but not necessarily diagonal. Make the objective

$(x-x_0)^TQ(x-x_0)$

In [17]:
#The following has a strong-enough diagonal that we're sure it is PSD
Q = np.random.random((n,n)) - .5 + 3*n*np.eye(n)

x0 = 10 * np.random.random(n) #creates a point with random coordinates in [0,10]

#minimized weighted distance between x and x0: (x-x0)' Q(x=x0)

obj2 = xp.Dot((x-x0), Q, (x-x0))

p.setObjective(obj2, sense = xp.minimize)
p.solve()

FICO Xpress v8.11.0, Community, solve started 8:39:38, Nov 24, 2020
Heap usage: 2652KB (peak 2652KB, 627KB system)
Minimizing QP noname with these control settings:
OUTPUTLOG = 1

Original problem has:
        20 rows           30 cols          600 elements
       900 qobjelem
Presolved problem has:
        20 rows           30 cols          600 elements
       900 qobjelem
Presolve finished in 0 seconds
Heap usage: 2685KB (peak 2685KB, 627KB system)

Coefficient range                    original                 solved        
  Coefficients   [min,max] : [ 5.58e-04, 10.00e-01] / [ 1.31e-04,  1.99e+00]
  RHS and bounds [min,max] : [ 6.08e-03,  9.81e-01] / [ 6.08e-03,  9.81e-01]
  Objective      [min,max] : [ 3.44e+01,  1.61e+03] / [ 4.30e+00,  7.41e+02]
  Quadratic      [min,max] : [ 4.25e-04,  1.81e+02] / [ 8.50e-04,  7.21e+02]
Autoscaling applied standard scaling

Barrier cache sizes : L1=32K L2=46080K
Using AVX support
Cores per CPU (CORESPERCPU): 2
Barrier starts after 0 seconds, u

Obviously we can use arrays of variables in constraints. Let's consider the following problem:

### Production planning with quadratic cost

A factory must plan production on one machine for the upcoming $N=12$ days. If, on one day, the machine is on, it must pay a fixed cost F, regardless of how much is produced on that machine. If the machine is on, the production must range between L and U. It must also pay a cost that is C times the square of the produced amount.

The demand at a given day is $w_i$, and the unsold product can be stored in a storage container of capacity $M$. Finally, the storage level at the end of the $N$ days must be the same as the beginning, which is set as $S$.

**Model**. Define two classes of variables, $x$ (continuous) and $y$ (binary), indicating the amount of production in a given day and whether the machine is on or off, respectively. Also, introduce a storage variable $s$ that is also indexed by the set of periods.

In [18]:
n = 12 #periods
N = range(n) 

C = 2 #proportional cost
L = 10 #minimum production level
U = 90 #maximum production level
F = 9000 #Fixed cost
M = 170 #maximum storage level
S = 25 #initial storage level

#demand
w = [60, 15, 25, 70, 70, 85, 10, 5, 65, 40, 50, 15]

#variables
x = [xp.var() for _ in N]
y = [xp.var(vartype = xp.binary) for _ in N]
s = [xp.var(ub=M) for _ in range(n+1)] #can assign the upper bound of M immediately

#total production cost (to be minimized)

objective = xp.Sum(F*y[i] + C*y[i]*x[i]**2 for i in N)

#y determines bounds on x
x_lb = [x[i] >= L*y[i] for i in N]
x_ub = [x[i] <= U*y[i] for i in N]

# storage at last period

store_end = s[n] == S
store_ini = s[0] == S

# Production balance

balance = [s[i] + x[i] == s[i+1] + w[i] for i in N]  # Conservation at each time period

p = xp.problem(x, y, s,
               objective,
               x_lb, x_ub, store_end, store_ini, balance)

p.solve()

print('time demand machine productn storage')
for i in N:
    print('{0:4d} {1:6d} {2:7.0f} {3:8.4f} {4:7.4f}'.format(i, w[i],
                                                            p.getSolution(y[i]),
                                                            p.getSolution(x[i]),
                                                            p.getSolution(s[i])))



Original problem size
   linear:    39 rows, 39 columns, 87 linear coefficients
   nonlinear: 1 coefficients, 95 tokens
Nonlinear presolve
   compressing formula space (in use\total : 96\713)
   removed 22 formulas and 616 formula tokens
   linear row reductions removed 2 rows, 2 columns, 4 linear coefficients
   bound tightening reduced 16 bounds
   objective transfer row objtransferrow through variable objtransfercol
Presolved problem size
   linear:    37 rows, 37 columns, 83 linear coefficients
   nonlinear: 1 coefficients, 95 tokens
Problem is nonlinear presolved
FICO Xpress v8.11.0, Community, solve started 8:39:38, Nov 24, 2020
Control settings used:
XSLP_POSTSOLVE = 1
XSLP_DELETIONCONTROL = 0
OUTPUTLOG = 1

Maximum expanded nl-formula size: 95  (row 'objtransferrow')
Total tokens: 95
  2  parallel calculation threads
  Jacobian: symbolic differentiation
           1 base AD formula, 96 average complexity
          24 in the Jacobian, 6 average complexity
Initial point objectiv

#Homework:
Rewrite the problem using NumPy for defining parameters, variables, constraints and objective.

In [19]:
#demand

W = np.array(w)

#variables

x = xp.vars(n, name = 'prod')
y = xp.vars(n, vartype = xp.binary, name = 'machine')
s = xp.vars(n + 1, ub = M, name = 'storage') 

#total production cost

objective = xp.Sum(F*y + C*x**2)

# y determines bounds on x

x_lb = x >= L*y
x_ub = x <= U*y

#storage at last period

store_end = s[n] == S
store_ini = s[0] == S

#production balance
balance = s[:n] + x == s[1:] + w #conservation at each time period

p = xp.problem(x,y,s,
               objective,
               x_lb,x_ub,store_end, store_ini, balance)

p.solve()

print('time demand machine productn storage')
for i in N:
    print('{0:4d} {1:6d} {2:7.0f} {3:8.4f} {4:7.4f}'.format(i, w[i],
                                                            p.getSolution(y[i]),
                                                            p.getSolution(x[i]),
                                                            p.getSolution(s[i])))


FICO Xpress v8.11.0, Community, solve started 8:39:52, Nov 24, 2020
Heap usage: 362KB (peak 362KB, 917KB system)
Minimizing MIQP noname with these control settings:
OUTPUTLOG = 1

Original problem has:
        38 rows           37 cols           86 elements        12 globals
        12 qobjelem
Presolved problem has:
        32 rows           32 cols           74 elements        11 globals
        12 qobjelem
LP relaxation tightened
Presolve finished in 0 seconds
Heap usage: 395KB (peak 402KB, 918KB system)

Coefficient range                    original                 solved        
  Coefficients   [min,max] : [ 1.00e+00,  9.00e+01] / [ 7.81e-03,  2.81e+00]
  RHS and bounds [min,max] : [ 1.00e+00,  1.70e+02] / [ 1.00e+00,  3.60e+02]
  Objective      [min,max] : [ 9.00e+03,  9.00e+03] / [ 3.50e+01,  9.00e+03]
  Quadratic      [min,max] : [ 4.00e+00,  4.00e+00] / [ 2.50e-01,  2.50e-01]
Autoscaling applied standard scaling

Will try to keep branch and bound tree memory usage below 11.8